# Deep-Dream Pytorch implementation

## Setup instructions
* Install Conda
https://www.anaconda.com/download/#macos

* Install pytorch and torchvision (computer vision for pytorch) package

`conda install pytorch torchvision -c pytorch`

https://pytorch.org/

## Overview
* https://ai.googleblog.com/2015/06/inceptionism-going-deeper-into-neural.html 
* https://en.wikipedia.org/wiki/DeepDream

## References
* https://github.com/sar-gupta/deep-dream-pytorch

### Python imports
* torch, numpy, torchvision
* PIL: Python Imaging Library
* pyplot: plotting image

In [55]:
from pathlib import Path
import glob
import torch
from torchvision import models, transforms
import numpy as np
from matplotlib import pyplot
%matplotlib inline
from PIL import Image, ImageFilter, ImageChops

Program configs

In [56]:
CUDA_ENABLED = False

# Deep dream configs
LAYER_ID = 28 # The layer to maximize the activations through
NUM_ITERATIONS = 5 # Number of iterations to update the input image with the layer's gradient
LR = 0.2

# We downscale the image recursively, apply the deep dream computation, scale up, and then blend with the original image 
# to achieve better result.
NUM_DOWNSCALES = 20
BLEND_ALPHA = 0.6

DeepDream class: use a vgg16 pretrained model

In [57]:
class DeepDream:
    def __init__(self, image):
        self.image = image
        self.model = models.vgg16(pretrained=False)
        if CUDA_ENABLED:
            self.model = self.model.cuda()
        self.modules = list(self.model.features.modules())
        
        # vgg16 use 224x224 images
        imgSize = 224
        self.transformMean = [0.485, 0.456, 0.406]
        self.transformStd = [0.229, 0.224, 0.225]
        self.transformNormalise = transforms.Normalize(
            mean=self.transformMean,
            std=self.transformStd
        )
        
        self.transformPreprocess = transforms.Compose([
            transforms.Resize((imgSize, imgSize)),
            transforms.ToTensor(),
            self.transformNormalise
        ])
        
        self.tensorMean = torch.Tensor(self.transformMean)
        if CUDA_ENABLED:
            self.tensorMean = self.tensorMean.cuda()

        self.tensorStd = torch.Tensor(self.transformStd)
        if CUDA_ENABLED:
            self.tensorStd = self.tensorStd.cuda()

    def toImage(self, input):
        return input * self.tensorStd + self.tensorMean

### Main deep dream algorithm
Input 
* an input image
* layerId
* number of iterations

Makes a forward pass until layerId, computes the gradient and updates the input image. This is repeated for the given number of iterations.

In [58]:
class DeepDream(DeepDream):
    def deepDream(self, image, layer, iterations, lr):
        transformed = self.transformPreprocess(image).unsqueeze(0)
        if CUDA_ENABLED:
            transformed = transformed.cuda()
        input = torch.autograd.Variable(transformed, requires_grad=True)
        self.model.zero_grad()
        for _ in range(iterations):
            out = input
            for layerId in range(layer):
                out = self.modules[layerId + 1](out)
            loss = out.norm()
            loss.backward()
            input.data = input.data + lr * input.grad.data

        input = input.data.squeeze()
        input.transpose_(0,1)
        input.transpose_(1,2)
        input = np.clip(self.toImage(input), 0, 1)
        return Image.fromarray(np.uint8(input*255))

Recursively apply deepDream at different scales and blend the result images to make the final image looks better.

In [59]:
class DeepDream(DeepDream):
    def deepDreamRecursive(self, image, layer, iterations, lr, num_downscales):
        if num_downscales > 0:
            # scale down the image
            image_small = image.filter(ImageFilter.GaussianBlur(2))
            small_size = (int(image.size[0]/2), int(image.size[1]/2))            
            if (small_size[0] == 0 or small_size[1] == 0):
                small_size = image.size
            image_small = image_small.resize(small_size, Image.ANTIALIAS)
            
            # run deepDreamRecursive on the scaled down image
            image_small = self.deepDreamRecursive(image_small, layer, iterations, lr, num_downscales-1)
            
            # Scale up the result image to the original size
            image_large = image_small.resize(image.size, Image.ANTIALIAS)
            
            # Blend the two image
            image = ImageChops.blend(image, image_large, BLEND_ALPHA)
        img_result = self.deepDream(image, layer, iterations, lr)
        img_result = img_result.resize(image.size)
        return img_result
    
    def deepDreamProcess(self):
        return self.deepDreamRecursive(self.image, LAYER_ID, NUM_ITERATIONS, LR, NUM_DOWNSCALES)

Now it will load each image and perform "deep dream" on it. Note that the computation could take a few minutes each

In [60]:
sourcesFolder = Path("./source_images")
dreamedFolder = Path("./dreamed_images")
jpgFilePaths = []

for filePath in sourcesFolder.glob("*.jpg"):
    jpgFilePaths.append(Path(filePath))

def dreamImage(jpgPath):
    image = Image.open(jpgPath)
    pyplot.imshow(image)
    pyplot.title("Image loaded from " + str(jpgPath))
    dreamedImage = DeepDream(image).deepDreamProcess()
    pyplot.imshow(dreamedImage)
    pyplot.title("Deep dreamed image")
    dreamedImage.save(dreamedFolder / ('deepdreamed-' + jpgPath.name))    

for path in jpgFilePaths:
    print("processing image: " + path.name)
    dreamImage(path)
    

processing image: deepdream_starry_night.jpg


AttributeError: 'PosixPath' object has no attribute 'absolutePath'